In [ ]:
import networkx as nx
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from node2vec import Node2Vec
from gensim.models import Word2Vec
import os
import pickle

pd.set_option('display.max_columns', None)


# Load the dataset
df_pueblos = pd.read_csv("../../data/interim/pueblos_recommender.csv")

pueblos_cmun = df_pueblos['cmun'].tolist()
user_files = os.listdir("../../data/user_output/")

user_choices = []

# Get all files from the user_output directory

# Read each file into a DataFrame and add to the list
for file in user_files:
    file_path = os.path.join("../../data/user_output/", file)
    user_df = pd.read_csv(file_path)
    user_choices.append(user_df)

# user choices of first user
user_choices_cmun = user_choices[0].cmun.tolist()
display(pueblos_cmun)
display(user_choices_cmun)

[48067,
 15064,
 15039,
 48062,
 48905,
 33038,
 33005,
 39067,
 31250,
 33078,
 39045,
 39069,
 39007,
 48087,
 48081,
 48060,
 31264,
 33001,
 31239,
 39100,
 48037,
 48007,
 33061,
 20901,
 48086,
 39081,
 48052,
 39066,
 33062,
 33048,
 39057,
 39088,
 39049,
 20003,
 39063,
 48030,
 33052,
 15027,
 39034,
 48915,
 31153,
 39101,
 48912,
 33003,
 15068,
 48008,
 20027,
 39078,
 20016,
 27015,
 39072,
 31082,
 15003,
 39098,
 31259,
 48022,
 48051,
 15040,
 15014,
 33027,
 39021,
 15001,
 27029,
 39026,
 48097,
 39082,
 48092,
 48055,
 27010,
 20002,
 39014,
 39003,
 33058,
 20028,
 31024,
 31022,
 20014,
 48039,
 33067,
 27053,
 15029,
 33050,
 20075,
 48058,
 48019,
 39083,
 20048,
 39004,
 33015,
 27022,
 48065,
 48042,
 48910,
 20006,
 31117,
 48095,
 39022,
 48094,
 20046,
 31226,
 20066,
 1042,
 20010,
 15024,
 39097,
 39086,
 20041,
 39089,
 39055,
 48006,
 39046,
 48025,
 27046,
 48050,
 24116,
 39015,
 33053,
 33072,
 48005,
 20023,
 27007,
 48026,
 20031,
 31087,
 24106,
 

[5116, 6016, 42039, 45052, 41901]

In [6]:
df_pueblos_enc = df_pueblos.filter(regex=r"^enc_", axis=1)
# Select only numerical features with more than 2 unique values (non-binary features)
pueblos_numerical_features = df_pueblos_enc.loc[:, (df_pueblos_enc.nunique() > 2)]

# Process user choices: filter for encoded features and remove binary columns
processed_user_choices = []
for df in user_choices:
    df_filtered = df.filter(regex=r"^enc_", axis=1)
    df_non_binary = df_filtered.loc[:, ~(df_filtered.isin([0.0, 1.0]).all())]
    processed_user_choices.append(df_non_binary)

display(processed_user_choices[0])
display(pueblos_numerical_features)

,enc_16,enc_17,enc_18,enc_19,enc_20,enc_21,enc_22,enc_23,enc_24,enc_25
0,-0.648114,-0.690213,-0.883675,0.721064,-0.729801,-1.645288,0.250512,-0.397119,1.024143,-0.634760
1,2.073570,0.176565,-0.883675,0.721064,-0.729801,0.910387,0.250512,0.884102,0.725820,2.056583
2,-0.537287,-0.688072,-0.883675,-1.242798,-0.988038,-0.596554,-0.336392,-0.397119,-1.616990,-0.545963
3,2.901560,0.595903,1.841297,0.721064,0.819623,1.512503,-0.336392,0.453842,1.111694,2.902572
4,2.024581,1.683101,1.062733,0.721064,0.044911,1.395117,0.250512,1.132697,1.139257,2.087258


,enc_16,enc_17,enc_18,enc_19,enc_20,enc_21,enc_22,enc_23,enc_24,enc_25
0,0.521190,1.185980,1.062733,0.721064,-0.213326,0.750134,1.424322,0.386912,0.224831,0.482462
1,1.199804,0.865604,0.284170,0.721064,2.110809,-0.135614,1.424322,0.731121,0.639890,1.260643
2,0.350131,0.279576,-0.105112,0.721064,1.077860,-0.236198,1.424322,-0.397119,-0.021610,0.371063
3,-0.413611,0.994147,0.284170,0.721064,-0.213326,0.732753,1.424322,-0.397119,0.299412,-0.413576
4,1.939453,2.815453,2.619860,0.721064,0.819623,0.890158,3.771941,6.343633,0.952805,1.977473
...,...,...,...,...,...,...,...,...,...,...
6665,-0.395943,1.096759,1.062733,0.721064,0.561385,-0.174132,0.250512,-0.397119,0.125931,-0.374828
6666,3.709472,7.126529,-0.494394,0.721064,-0.729801,1.572083,0.250512,2.413918,1.029007,3.574197
6667,0.558132,1.842909,0.284170,0.721064,1.077860,0.602008,0.250512,0.511210,0.578280,0.532511
6668,3.480590,3.239257,-0.883675,0.721064,-0.471564,1.345819,0.250512,2.413918,0.552338,3.398218


In [ ]:
# 1️⃣ Construct the Graph
G = nx.Graph()
for i, row in pueblos_numerical_features.iterrows():
    # get cmun of row
    cmun = df_pueblos.iloc[i, 0]
    G.add_node(cmun, **row.to_dict())

# Compute pairwise feature similarity (Cosine Similarity)
feature_matrix = pueblos_numerical_features  # drop non numeric columns
similarity_matrix = cosine_similarity(feature_matrix)

# Add edges based on similarity threshold
threshold = 0.75  # Adjust threshold based on data distribution
for i in range(len(feature_matrix)):
    for j in range(i + 1, len(feature_matrix)):
        if similarity_matrix[i, j] > threshold:
            G.add_edge(i, j, weight=similarity_matrix[i, j])

In [ ]:
# 2️⃣ Generate Node2Vec Embeddings
node2vec = Node2Vec(G, dimensions=20, walk_length=10, num_walks=20, workers=6)


In [ ]:
model = node2vec.fit(window=5, min_count=1, batch_words=4)

In [ ]:
display(similarity_matrix)

In [ ]:
# Create models directory if it doesn't exist
os.makedirs('../../models', exist_ok=True)

# Save the Node2Vec embeddings
model_path = '../../models/node2vec_embeddings.model'
model.save(model_path)

print(f"Model saved to {model_path}")

# If you want to save the graph separately
graph_path = '../../models/village_graph.pkl'
with open(graph_path, 'wb') as f:
    pickle.dump(G, f)
    
print(f"Graph saved to {graph_path}")

Model saved to ../../models/node2vec_embeddings.model
Graph saved to ../../models/village_graph.pkl


In [ ]:
# 3️⃣ Compute Similarity
user_selected_villages = user_choices_cmun # Example selected villages
selected_vectors = [model.wv[str(v)] for v in user_selected_villages]
centroid_vector = np.mean(selected_vectors, axis=0)

# Rank all villages based on cosine similarity
all_village_vectors = {v: model.wv[str(v)] for v in G.nodes()}
similarities = {v: cosine_similarity([centroid_vector], [vec])[0][0] for v, vec in all_village_vectors.items()}
ranked_villages = sorted(similarities.items(), key=lambda x: x[1], reverse=True)

# 4️⃣ Evaluate Ranking Quality
def ndcg_at_k(ranked_list, ideal_list, k):
    def dcg(scores):
        return sum([(2**s - 1) / np.log2(i+2) for i, s in enumerate(scores)])
    
    ideal_scores = [1 if v in ideal_list else 0 for v, _ in ranked_list[:k]]
    return dcg(ideal_scores) / dcg(sorted(ideal_scores, reverse=True))

ndcg_score = ndcg_at_k(ranked_villages, user_selected_villages, k=10)
print(f"NDCG Score: {ndcg_score:.4f}")

# Display top similar villages
print("Top 10 Most Similar Villages:")
for village, score in ranked_villages[:10]:
    print(f"Village {village}: Similarity {score:.4f}")


No matching villages found in the graph. User choices may not be in the dataset.
NDCG Score: nan
Top 10 Most Similar Villages:
Village 0: Similarity 0.0000
Village 1: Similarity 0.0000
Village 2: Similarity 0.0000
Village 3: Similarity 0.0000
Village 4: Similarity 0.0000
Village 5: Similarity 0.0000
Village 6: Similarity 0.0000
Village 7: Similarity 0.0000
Village 8: Similarity 0.0000
Village 9: Similarity 0.0000


/var/folders/w9/bt5d4t050m7fd91t2r7l1mlh0000gn/T/ipykernel_83209/889445542.py:30: RuntimeWarning: invalid value encountered in scalar divide
  return dcg(ideal_scores) / dcg(sorted(ideal_scores, reverse=True))
